# DSCB310 – Datenanalyse und Business Intelligence 1 – Übungsblatt 2 – WS 2022/23 

Matrikelnummern: 80720, 67915, 79713

## Vorverarbeitung der Daten

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as msn
import seaborn as sns
import json
import warnings
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from plotly.subplots import make_subplots

pfad = ("Shopping_Carts.parquet")
df = pd.read_parquet(pfad)

df.reordered = df.reordered.astype(bool)

days = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['order_dow'] = df['order_dow'].map(days)

df.tip = df.tip.astype(bool)

In [ ]:
with open('california-counties.geojson') as response:
    counties = json.load(response)

## Explorative Datenanalyse

In [ ]:
print(f'Der Datensatz enthält {df.order_id.nunique()} unterschiedliche Bestellungen, von {df.user_id.nunique()} verschiedenen Usern.')
print(f'Es wurden {df.product_id.nunique()} Produkte verkauft, aus {df.department_id.nunique()} Departments die in {df.aisle_id.nunique()} Aisles aufgeteilt sind.')
print(f'Die Produkte wurden in {df.county.nunique()} Countys in Californien verkauft.')

In [ ]:
px.bar(df.groupby(["order_dow", 'tip'], as_index=False)[["order_id"]].count().rename(columns={"order_id" : "Anzahl"}), 
       x="order_dow", y="Anzahl", 
       category_orders={"order_dow" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, 
       color='tip',
       labels={'order_dow': 'Tag der Bestellung', 'Anzahl': 'Anzahl an Bestellungen'},
       title='Montags und dienstags die meisten Bestellungen'
      )

In [ ]:
tab = pd.crosstab(df['order_hour_of_day'], df['order_dow'])
px.line(tab, labels={'order_hour_of_day': 'Stunden am Tag', 'value': 'Anzahl an Bestellungen'}, title='Peak um die Mittagsstunden')

## Aufgaben Personalabteilung

In [ ]:
tab_viz = pd.crosstab(df['order_dow'], df['tip'], normalize='index')

fig = px.bar(tab_viz, barmode='group', 
             category_orders={"order_dow" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]},
             labels={'order_dow': 'Tag der Bestellung', 'value': 'Anteil'},
             title='Montags und dienstags geben Kunden mehr Trinkgeld'
            )

fig.show()

In [ ]:
df_viz_tip = pd.DataFrame(pd.crosstab([df['department'], df['aisle']], df['tip'], normalize='index').sort_values(by=True, ascending=False)).reset_index()
px.treemap(df_viz_tip,
           path=['department', 'aisle'], color=True,
           height=500, title='Bei Bestellungen mit Alkohol gab es mehr Trinkgeld'
           )

In [ ]:
tab = pd.crosstab(df['order_hour_of_day'], df['tip'], normalize='index')
px.bar(tab, labels={'order_hour_of_day': 'Stunden am Tag', 'value': 'Anteil'}, title='Mehr Trinkgeld bei frühen/späten Bestellungen')

### P.1 Zusammenhang zwischen Produkten (24852, 24964 und 2120) in einer Bestellung und Trinkgeldverhalten

#### Produkt Nummer 24852: Bananen (fresh fruits / Obst)

In [ ]:
dfproduct24852 = df[df["product_id"]==24852]
pd.crosstab(dfproduct24852['tip'], dfproduct24852["product_id"], normalize='columns')

In [ ]:
px.bar(pd.crosstab(dfproduct24852['tip'], dfproduct24852["product_id"], normalize='columns'))

In [ ]:
dfaisleff=df[df["aisle"]=="fresh fruits"]
dfaisleff[["aisle","tip"]].groupby("aisle").mean()

Schaut man sich die Tip-Rate für Bestellungen an, bei denen Bananen dabei sind, so fällt auf, dass im Vergleich zur durchschnittlichen Trinkgeldwahrscheinlichkeit über alle Produkte (45%) die Trinkgeldrate schon etwas angestiegen ist, nämlich auf 53%. <br> <br> Nun schauen wir uns noch an, wie viel Prozent die Produkte durchschnittlich bekommen, die zur Kategorie "fresh fruits" gehören. Dabei fällt auf, dass der Durchschnitt hier 47% beträgt.<br><br> Es lässt sich also sagen, dass alleine durch Produkte aus der Kategorie "fresh fruits" die Trinkgeldwahrscheinlichkeit steigt. Bestellt der Kunde zusätzlich noch das Produkt "Banane", so steigt die Tip-Rate erneut ein bisschen an

#### Produkt Nummer 24964: Organic Garlic (Knoblauch) (fresh vegetables / Gemüse)

In [ ]:
df24964 = df[df["product_id"]==24964]
pd.crosstab(df24964['tip'], df24964["product_id"], normalize='columns')

In [ ]:
px.bar(pd.crosstab(df24964['tip'], df24964["product_id"], normalize='columns'))

In [ ]:
dfaislefv=df[df["aisle"]=="fresh vegetables"]
dfaislefv[["aisle","tip"]].groupby("aisle").mean()

Bei dem Produkt Organic Garlic fällt zuerst einmal auf, dass die Trinkgeldwahrscheinlichkeit, wenn das Produkt sich in einer Bestellung findet, ein bisschen niedriger ist (42%) als der Durchschnitt über alle Produkte (45%). <br><br> Auch hier haben wir uns erneut angeschaut, wie das Tip-Verhalten ist, wenn ein Produkt aus der Kategorie "fresh vegetables" sich in der Bestellung eines Kunden befindet. Dabei fällt auf, dass sich die Wahrscheinlichkeit von Trinkgeld nicht verändert hat und immer noch bei ca. 42% liegt. <br><br> Daraus lässt sich schließen, dass sobald ein Kunde ein Produkt aus der Kategorie "fresh vegetables" bestellt, sich die Trinkgeldwahrscheinlichkeit etwas verringert. Bestellt er dann zusätzlich noch "Organic Garlic" verändert sich diese Wahrscheinlichkeit jedoch nicht mehr.

#### Produkt Nummer 2120: Sauvignon Blanc (Wein) (department alcohol)

In [ ]:
df2120 = df[df["product_id"]==2120]
pd.crosstab(df2120['tip'], df2120["product_id"], normalize='columns')

In [ ]:
px.bar(pd.crosstab(df2120['tip'], df2120["product_id"], normalize='columns'))

In [ ]:
dfa=df[df["aisle"]=="white wines"]
dfa[["aisle","tip"]].groupby("aisle").mean()

In [ ]:
#Chi-Quadrat Versuch
df2 = df.copy()
df2['new_col'] = np.where(df['product_id']==2120,'2120', 'Other')
pd.crosstab(df2['tip'], df2["new_col"])
# Zählen der Anzahl von Trinkgeld-Gebern und Nicht-Trinkgeld-Gebern
observed_values = pd.crosstab(index=df2['tip'], columns=df2["new_col"])

# Durchführung des chi-Quadrat-Tests
chi2, p, dof, expected_values = chi2_contingency(observed_values)

if p < 0.01:
    print(f"Hoch Signifikant\np-Wert: {p}")
elif p < 0.5:
    print(f"Signifikant\np-Wert: {p}")
else:
    print(f"Nicht signifikant\np-Wert: {p}")

Als wir uns das Produkt Sauvignon Blanc angesehen haben, ist uns ein enormer Sprung bei der Trinkgeldwahrscheinlichkeit auf 84% aufgefallen. Daraufhin haben wir überprüft, ob das Produkt einfach zu wenige Bestellungen hat und dadurch eine so hohe Tip-Rate entstanden ist. Jedoch wurde das Produkt insgesamt um die 1.500 mal bestellt und somit kann man hier nicht mehr von reinem Zufall sprechen. Zu dem wurde das Ergebnis mit mehreren Samples der Grundmenge erneut durchgeführt und nur leichte schwankungen von 0,5% bis 2% beobachtet. <br><br> Auch hier wurde dies dann wieder mit der Wahrscheinlichkeit für die Kategorie (Weiße Weine) verglichen, wobei diese eine Tip-Rate von 84% hat. Auch das Department "Alkohol" wurde überprüft und auch dieses hat eine sehr hohe 82% Wahrscheinlichkeit für Trinkgeld. <br><br> Abschließend kann man sagen, sobald der Kunde ein Produkt aus dem Department "Alkohol" oder der Kategorie "Weiße Weine" bestellt, steigt die Trinkgeldwahrscheinlichkeit sehr stark an. Wird anschließend noch das Produkt Sauvignon Blanc dazu bestellt, ändert sich die bereits sehr hohe Trinkgeldwahrscheinlichkeit (82%) nur ein bisschen, auf 84%

### P.2: Regionale Unterschiede beim Trinkgeldverhalten

In [ ]:
tip_per_county = pd.crosstab(df['county'], df['tip'], normalize='index')
tip_per_county = tip_per_county.reset_index()
tip_per_county = tip_per_county.drop(axis=1, labels=[False], errors='ignore')
df = df.merge(tip_per_county, left_on='county', right_on='county')
df = df.rename(columns={True: 'tip_rate'})

In [ ]:
fig = px.choropleth_mapbox(data_frame=df, geojson=counties, 
                           featureidkey='properties.name', locations='county',
                           color='tip_rate',
                           mapbox_style="carto-positron",
                           zoom=4.2, center = {"lat":  37.17571858166602, "lon": -119.41580973739858},
                           height=500,
                           labels={'tip_rate': 'Trinkgeld-Wahrscheinlichkeit'},
                           color_continuous_scale="RdYlGn"
                          )
fig.show()

In [2]:
dff = df[df['tip']==False]
x = ((df[['county','tip']].groupby("county").count() - dff[['county','tip']].groupby("county").count()) / df[['county','tip']].groupby("county").count()).rename(columns={"tip":"tipPercentage"})
y = df[["order_id","county"]].groupby("county").count().rename(columns={"order_id":"orderCount"})
dfTipCountCounty = x.merge(y, how="inner", on="county")
dfTipCountCounty = dfTipCountCounty.sort_values(by="tipPercentage", ascending=False)

In [3]:
fig1 = px.line(dfTipCountCounty.reset_index(), y="tipPercentage", x="county")
fig2 = px.bar(dfTipCountCounty.reset_index(), y="orderCount", x="county")
fig2.update_traces(yaxis="y2")
fig = make_subplots(specs=[[{"secondary_y":True}]])
fig.update_yaxes(title_text="Trinkgeldanteil", secondary_y=False, color="red")
fig.update_yaxes(title_text="Bestellhäufigkeit", secondary_y=True, color="blue")
fig.add_traces(fig1.data + fig2.data)
fig.update_traces(line=dict(color="Red"), secondary_y=False)
fig.update_traces(marker=dict(color="Blue"), secondary_y=True)
fig.update_layout(title_text="Starke Schwankung bei Trinkgeldvergabe nicht allein zurückzuführen auf Anzahl Bestellungen")
fig.show()

Es lassen sich teilweise starke bis sehr starke Schwankungen bei den durchschnittlichen Trinkgeldwahrscheinlichkeiten der einzelnen Counties erkennen. Jedoch, entgegen der Hypothese, dass dies auf die unterschiedliche Menge an Bestellungen pro County zurückzuführen ist, gibt es nur bei einem County einen deutlichen Rückgang der Tip-Rate, wo gleichzeitig auch sehr wenige Bestellungen getätigt wurden. Bei den anderen Counties, ist nämlich auch eine deutliche Schwankung der Trinkgeldraten zu sehen. Zwischen 56% und 34%, obwohl die Anzahl an Bestellung bei über ca. 10.000 liegen. <br> <br>Obwohl in manchen Counties die niedrige Anzahl an Bestellungen diese Schwankungen verursacht haben könnten, gibt es eine vielzahl an anderen Counties, bei denen dieses Verhalten, auch wenn nicht so extrem, auch aufgetreten ist. Somit muss es an anderen Faktoren liegen, weshalb die Trinkgeldwahrscheinlichkeiten regional so unterschiedlich sind

### P.3: Welche Attribute wirken sich auf das Trinkgeldverhalten aus?

In [ ]:
#Erstellung einer Korrelationsmatrix 
corrMa = df.corr()
mask = np.array(corrMa)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
ax.set_title("")
sns.heatmap(corrMa,mask=mask,vmax=.8, square=True, annot=True)

In [ ]:
corrplot = corrMa[8:-3]
corrplot = corrplot.drop(['tip', "tip_rate"], axis=1)
corrplot = corrplot.T

fig = px.bar(corrplot, orientation="h", labels={"index":"Attribute", "value":"Auswirkung", "variable":"Variable"}, title="Korrelation von \"days_since_prior_order\" mit \"tip\" am stärksten")
fig.show()

Bei den Attributen, die die Trinkgeldwahrscheinlichkeit beeinflussen, gibt es gleich mehere die diese positiv Verändern: <br> - add_to_cart_order: Steigt diese Zahl an, also hat der Kunde in einer Bestellung viele Produkte so steigt die Wahrscheinlich für Trinkgeld. <br> - reordered: Bestellt der Kunde erneut ein Produkt, beziehungsweiße auch mehrere Produkte, welches er schonmal gekauft hat, so steigt auch hier die Wahrscheinlich für Trinkgeld. <br> - order_number: Wenn der Kunde bereits öfter bestellt, die Anzahl an Bestellungen des Kunden insgesamt also hoch, so beeinflusst dies ebenso positiv die Wahrscheinlich für Trinkgeld. <br><br> Ebenso gibt es aber auch Attribute die die Trinkgeldwahrscheinlichkeit verschlechtern: <br> - days_since_prior_order: Je länger die letzte Bestellung eines Kunden her ist, desto niedriger wird auch die Trinkgeldwahrscheinlich bei diesem Kunden, sollte er erneut etwas bestellen.

### P.4: Spielt die Vergangenheit eines Users eine Rolle bei der Trinkgeldwahrscheinlichkeit?

![alt text](Bilder\outputCut.png "Screenshot")

In [ ]:
X = df[["user_id","order_number","tip"]].groupby(["user_id", "order_number","tip"],as_index=False).count()
X["cumsum"] = X.groupby(["user_id"])["tip"].transform(pd.Series.cumsum)
df = df.merge(X)
df["TipHistory"] = df["cumsum"] / df["order_number"]
df = df.drop('cumsum', axis=1)

In [ ]:
#define used DataFrames
Y = df[["TipHistory","order_number","user_id"]].groupby(["order_number"], as_index=False).mean()
Y2 = df[["TipHistory","order_number","user_id"]].groupby(["order_number"], as_index=False).count()

#define colors to use
col1 = 'steelblue'
col2 = 'red'

#define subplots
fig,ax = plt.subplots(figsize=(15,8),)

#add first line to plot
ax.plot(Y.order_number, Y.TipHistory, color=col1)

#add x-axis label
ax.set_xlabel('order_number', fontsize=12, )

#add y-axis label
ax.set_ylabel('Tip', color=col1, fontsize=12)

#Horizontal Line (TipRate Average (All orders))
l = ax.axhline(y=0.447496, color='g', linestyle='-')

#define second y-axis that shares x-axis with current plot
ax2 = ax.twinx()

#add second line to plot
ax2.plot(Y.order_number, Y2.user_id, color=col2)

#add second y-axis label
ax2.set_ylabel('Anzahl Bestellungen', color=col2, fontsize=12)

#add a title
plt.title('Durschnittliche Trinkgeldwahrscheinlichkeit sinkt nach dem Peak bei Usern mit mehr als 39 Bestellungen ')

ax.text(x=0.5, y=0.5, s='AVG tip rate (Über alle Daten)', fontsize=13, color='g', horizontalalignment='right', verticalalignment='center', transform=ax.transAxes)

ax.set_xticks(range(0,105,5))

Ob die Vergangenheit des Nutzers oder die Parameter der Bestellung eine größere Rolle bei der bestimmung des Trinkgeld spielt wurde mit der Hilfe eines Machine Learning Modells überprüft. Es wurde ein Random Forest mit allen Spalten (außer 'product_name', 'department', 'aisle' lediglich ihre *_id Spalten wurden verwendet). Zusätzlich wurde der Prozentuale Anteil an erneut bestellten Produkten pro Bestellung eines Users und pro User insgesamt berechnet und dem Modell gelernt. Mit 80% Accuracy auf den Testdaten waren unter den Top 5 der wichtigsten Features: Reorderanteil des Users, die User ID und die Zeit der letzten Bestellung eines Users. Die tatsächlichen Parameter einer Bestellung waren für das Modell eher unwichtig und gering. <br><br> Daraus schließen wir, dass die tatsächlichen Parameter einer Bestellung nicht so wichtig sind, wie die Vergangen Daten des Users. <br><br>**Der Code für das Maschine-Learning-Model befindet sich am Ende des Notebooks** 

## Aufgaben Dispositon

In [ ]:
countyCount_vis = df.groupby('county')[['product_id']].count().sort_values(by='product_id', ascending=False).reset_index().rename(columns={'product_id': 'count_prod_per_county'})
fig = px.choropleth_mapbox(data_frame=countyCount_vis, geojson=counties, 
                           featureidkey='properties.name', locations='county',
                           color='count_prod_per_county', color_continuous_scale=px.colors.sequential.Brwnyl,
                           mapbox_style="carto-positron",
                           zoom=4.2, center = {"lat":  37.17571858166602, "lon": -119.41580973739858},
                           height=500,
                           labels={'count_prod_per_county': 'Anzahl gekaufter Produkte pro County'}
                          )
fig.show()

In [ ]:
df_product_count_s = df.groupby(['county', 'product_id', 'product_name'], as_index=False)[['order_id']].count()
df_product_count_s = df_product_count_s.rename(columns={'order_id': 'product_count'})
df_product_count_s['ranking_in_products'] = df_product_count_s.groupby('county')[['product_count']].rank(method='dense', ascending=False)

In [ ]:
top5counties = df_product_count_s[(df_product_count_s.county.isin(['Calaveras', 'Orange', 'Glenn', 'Fresno', 'Kern']))&(df_product_count_s.ranking_in_products.isin([1.0, 2.0, 3.0]))].sort_values(by=['county', 'ranking_in_products'])

In [ ]:
fig = px.bar(top5counties, x='ranking_in_products', y='product_count', 
             color='product_name', facet_col='county',
             category_orders={'ranking_in_products': [2.0, 1.0, 3.0]},
             text='product_name',
             color_discrete_sequence=['yellow', 'orange', 'red', 'green'],
             labels={'ranking_in_products': 'Rang', 'product_count': 'Anzahl an Produkten'},
             title='Top3-Produkte in den fünf stärksten Counties'
            )

fig.show()

In [ ]:
df_department_count_s = df.groupby(['county', 'department_id', 'department'], as_index=False)[['order_id']].count()
df_department_count_s = df_department_count_s.rename(columns={'order_id': 'department_count'})
df_department_count_s['ranking_in_departments'] = df_department_count_s.groupby('county')[['department_count']].rank(method='dense', ascending=False)

In [ ]:
top5counties_d = df_department_count_s[(df_department_count_s.county.isin(['Calaveras', 'Orange', 'Glenn', 'Fresno', 'Kern']))&(df_department_count_s.ranking_in_departments.isin([1.0, 2.0, 3.0]))].sort_values(by=['county', 'ranking_in_departments'])

In [ ]:
fig = px.bar(top5counties_d, x='ranking_in_departments', y='department_count', 
             color='department', facet_col='county',
             category_orders={'ranking_in_products': [2.0, 1.0, 3.0]},
             text='department',
             labels={'ranking_in_departments': 'Rang', 'department_count': 'Anzahl an Produkten'},
             title='Top3-Departments in den fünf stärksten Counties'
            )

fig.show()

In [ ]:
df_aisle_count_s = df.groupby(['county', 'aisle_id', 'aisle'], as_index=False)[['order_id']].count()
df_aisle_count_s = df_aisle_count_s.rename(columns={'order_id': 'aisle_count'})
df_aisle_count_s['ranking_in_aisles'] = df_aisle_count_s.groupby('county')[['aisle_count']].rank(method='dense', ascending=False)

In [ ]:
top5counties_d = df_aisle_count_s[(df_aisle_count_s.county.isin(['Calaveras', 'Orange', 'Glenn', 'Fresno', 'Kern']))&(df_aisle_count_s.ranking_in_aisles.isin([1.0, 2.0, 3.0]))].sort_values(by=['county', 'ranking_in_aisles'])

In [ ]:
fig = px.bar(top5counties_d, x='ranking_in_aisles', y='aisle_count', 
             color='aisle', facet_col='county',
             category_orders={'ranking_in_aisles': [2.0, 1.0, 3.0]},
             text='aisle',
             labels={'ranking_in_aisles': 'Rang', 'aisle_count': 'Anzahl an Produkten'},
             title='Top3-Aisles in den fünf stärksten Counties'
            )

fig.show()

### D.1: Prüfen Sie, ob es beim Produkt 13176 auffällige Muster beim Wiederbestellverhalten gibt.

In [ ]:
df_product_size = df.groupby("product_id", as_index=False).size().rename(columns={"size":"Anzahl"})
df_product_size["total_product_rank"] = df_product_size["Anzahl"].rank(method="dense", ascending=False)
df_product_size = df_product_size.sort_values("total_product_rank").merge(df[["product_id", "product_name"]].drop_duplicates())

In [ ]:
print("Produktname:", df.loc[df["product_id"] == 13176]["product_name"].unique()[0])
print(round(len(df.loc[df["product_id"] == 13176, "product_id"]) / len(df.product_id) * 100, 2), "%", "aller bestellten Produkte sind das Produkt \"Bag of organic Bananas\" und somit Platz", int(df_product_size.loc[df_product_size["product_id"] == 13176, "total_product_rank"].max()), "der meistverkauften Produkte")
print(round(df.loc[df["product_id"] == 13176].order_id.nunique() / df.order_id.nunique() * 100, 2), "%", "aller Bestellungen enthalten das Produkt \"Bag of organic Bananas\"")
print("Liste der Counties in denen das Produkt nicht bestellt wurde: ", set(df.county.unique()) - set(df.loc[df["product_id"] == 13176].county.unique()))

In [ ]:
df.loc[df["product_id"] == 13176][["department", "aisle"]].drop_duplicates()

In [ ]:
fig = px.bar(df.loc[df["product_id"] == 13176].groupby("reordered", as_index=False).size(), x="reordered", y="size", labels={"size":"Anzahl", "reordered":"Wiederbestellung"}, title="Bestellungen von Produkt 13176 sind meist Wiederbestellung")
gesamt_zahl = df.loc[df["product_id"] == 13176, "product_id"].count()
false_zahl = df.loc[(df["product_id"] == 13176) & (df["reordered"] == False), "product_id"].count()
false_anteil = round(false_zahl / gesamt_zahl * 100, 2)
true_zahl = df.loc[(df["product_id"] == 13176) & (df["reordered"] == True), "product_id"].count()
true_anteil = round(true_zahl / gesamt_zahl * 100, 2)
fig.add_annotation(x="false", y=false_zahl, text=f"{false_zahl} ({false_anteil}%)")
fig.add_annotation(x="true", y=true_zahl, text=f"{true_zahl} ({true_anteil}%)")
fig.show()

In [ ]:
df_anfrage = df.loc[df["product_id"] == 13176].groupby(["add_to_cart_order", "reordered"], as_index=False).size()
fig = px.line(df_anfrage.loc[df_anfrage["size"] >= 20], x="add_to_cart_order", y="size", color="reordered", labels={"size":"Anzahl", "add_to_cart_order":"Warenkorbposition"}, title="Hoher Anteil niedriger Warenkorbpositionen")
fig.add_vline(df.loc[df["product_id"] == 13176, "add_to_cart_order"].mean(), annotation={"text":"Mittelwert"})
fig.show()

In [ ]:
str_anteil = round(df.loc[df["product_id"]==13176, "tip"].mean(), 3)
fig = px.bar(pd.crosstab(df.loc[df["product_id"]==13176, "reordered"], df.loc[df["product_id"]==13176, "tip"], normalize="index").rename(columns={False:"Nix", True:"True"})[["True"]], title=f"Trinkgeldanteil von Produkt \"Bag of organic Bananas\": {str_anteil}, Wiederbestellung erhöht den Anteil des Trinkgelds", labels={"reordered":"Wiederbestellung", "value":"Anteil"})
str_anteil = round(df.tip.mean(), 3)
fig.add_hline(df.tip.mean(), annotation={"text":f"Durchschnittlicher Trinkgeldanteil auf allen Daten ({str_anteil})"}, annotation_position="top left")
y_value = round((df.loc[(df["product_id"]==13176) & (df["reordered"] == True), "tip"].mean()), 3)
fig.add_annotation(x="true", y=y_value, text=f"{y_value}")
fig.show()

In [ ]:
px.bar(df.loc[df["product_id"]==13176].groupby(["order_dow", "reordered"], as_index=False).size(), x="order_dow", y="size", facet_col="reordered", labels={"size":"Anzahl", "order_dow":"Wochentag"}, category_orders={"order_dow" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]})

In [ ]:
# Dauert viel zu lange (30min +)
# df["prior_order_13176"] = False
# for index, row in df.loc[df["product_id"] == 13176][["user_id", "order_number", "prior_order_13176"]].drop_duplicates().sort_values(["user_id", "order_number"]).iterrows():
#     df.loc[(df["user_id"] == row.user_id) & (df["order_number"] == row.order_number + 1), "prior_order_13176"] = True
# df.to_csv("priororder_df_output.csv")

df_ = pd.read_csv("priororder_df_output.csv", index_col="index")

In [ ]:
px.histogram(df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)], x= "days_since_prior_order", title="Wartezeit zwischen Folgebestellungen die beide Produkt 13176 enthalten")

In [ ]:
px.bar(df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)].groupby(["order_dow"], as_index=False)[["order_id"]].count().rename(columns={"order_id" : "Anzahl"}), x="order_dow", y="Anzahl", category_orders={"order_dow" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}, title="Bestellungen, wenn vorherige Bestellung Produkt 13176 enthielt auf Wochentage")


In [ ]:
px.histogram(df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)], x = "order_hour_of_day")

In [ ]:
pd.crosstab(df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)].product_id, df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)].tip, normalize="index")

In [ ]:
print("Counties in denen Produkt 13176 nie in 2 Folgebestellungen aufgetreten ist:\n", set(df_.county.unique()) - set(df_.loc[(df_.prior_order_13176) & (df_["product_id"] == 13176)].county.unique()))

### D.2: Spielen die Artikel aus dem Department „produce“ in allen Counties eine gleich große Rolle?

In [ ]:
print("Anzahl verschiedener Produkte:", df.product_id.nunique())
print("Anzahl verschiedener Produkte aus dem Department \"produce\":", df.loc[df["department"] == "produce"].product_id.nunique())
print("Anteil:", round(100 * df.loc[df["department"] == "produce"].product_id.nunique() / df.product_id.nunique(), 2), "%")

In [ ]:
print(round(df.loc[df["department"] == "produce"].order_id.nunique() / df.order_id.nunique() * 100, 2), "%", "der Bestellungen enthält ein Produkt aus dem Department \"Produce\"")

In [ ]:
fig = px.histogram(df.loc[df["department"] == "produce"].groupby(["order_id"], as_index=False).size(), x="size", title="Anzahl an Produce-Artikeln pro Bestellungen (mit mindestens 1 Artikel)", labels={"size":"Anzahl Produce-Artikel"})
mw = df.loc[df["department"] == "produce"].groupby(["order_id"], as_index=False).size()["size"].mean()
fig.add_vline(mw, annotation={"text":f"Mittelwert: {round(mw, 1)}"})
fig.update_yaxes(title_text="Anzahl Bestellungen")

In [ ]:
df = df.merge(df.groupby(["county"], as_index=False).size().rename(columns={"size":"order_size_per_county"}))
df = df.merge(df.loc[df.department == "produce"].groupby(["county"], as_index=False).size().rename(columns={"size":"order_size_produce_per_county"}))
df["relative_produce_per_county"] = round(df.order_size_produce_per_county / df.order_size_per_county * 100, 2)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
df_anteil = df.groupby(["county", "relative_produce_per_county"], as_index=False).size().sort_values(["relative_produce_per_county"], ascending=False)
df_anzahl = df.groupby(["county", "order_size_produce_per_county"], as_index=False).size().sort_values(["order_size_produce_per_county"], ascending=False)
fig1 = px.line(df_anteil, x="county", y="relative_produce_per_county")
fig2 = px.bar(df_anzahl, x="county", y="order_size_produce_per_county")
fig2.update_traces(yaxis="y2")
fig.update_yaxes(title_text="Anteil an allen Bestellungen", secondary_y=False, color="red")
fig.update_yaxes(title_text="Anzahl an Bestellungen", secondary_y=True, color="blue")
fig.add_traces(fig1.data + fig2.data)
fig.update_traces(line=dict(color="Red"), secondary_y=False)
fig.update_traces(marker=dict(color="Blue"), secondary_y=True)
fig.update_layout(title_text="Bestellungen von Produkten aus dem Department \"produce\"")
fig.show()

In [ ]:
df_department_count = df.groupby(["county", "department"], as_index=False).size().rename(columns={"size":"order_per_department_per_county"})
df_department_count.groupby("county")[["order_per_department_per_county"]]
df_department_count["department_rank"] = df_department_count.groupby("county")[["order_per_department_per_county"]].rank(method="dense", ascending=False)
px.sunburst(df_department_count.loc[df_department_count["department"] == "produce"].merge(df[["county", "department", "order_size_per_county"]].drop_duplicates()), path=["department_rank", "county", "order_size_per_county"])

In [ ]:
df_temp = df_department_count.loc[df_department_count["department"] == "produce"].merge(df[["county", "department", "order_size_per_county"]].drop_duplicates())
counties_temp = df_temp.loc[df_temp["department_rank"] == 2, "county"]
df_department_count.loc[(df_department_count["department_rank"] <= 2) & (df_department_count["county"].isin(counties_temp))]

In [ ]:
px.sunburst(df_department_count.loc[df_department_count["department"] == "dairy eggs"].merge(df[["county", "department", "order_size_per_county"]].drop_duplicates()), path=["department_rank", "county", "order_size_per_county"])

In [ ]:
fig = px.choropleth_mapbox(data_frame=df_anteil, geojson=counties, 
                           featureidkey='properties.name', locations='county',
                           color='relative_produce_per_county',
                           mapbox_style="carto-positron",
                           zoom=4.2, center = {"lat":  37.17571858166602, "lon": -119.41580973739858},
                           height=500,
                           labels={'relative_produce_per_county': 'Produce-Artikel Anteil'},
                           title="Keine räumliche Systematik erkennbar",
                           color_continuous_scale="RdYlGn"
                          )
fig.show()

### D.3: Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte? 

In [ ]:
# DataFrame mit allen Counties und der Anzahl verkaufter Produkte pro County
county_count = pd.DataFrame(df['county'].value_counts()).reset_index().rename(columns={'index': 'county', 'county': 'count'})

In [ ]:
# Liste mit allen Counties zum Iterieren
counties_i = county_count['county'].unique().tolist()

# Liste mit den Top10-Produkten für jedes County
county_list_top10 = []
for county in counties_i:
    county = df_product_count_s[(df_product_count_s['county']==county)&(df_product_count_s['ranking_in_products'].isin(np.arange(1,11)))].sort_values(by='ranking_in_products').reset_index().drop(axis=1, labels='index')
    county_list_top10.append(county)
    
# Liste, um Länge der Top10-Listen festzustellen (Länge > 10 häufig bei Counties mit wenigen verkauften Produkten)
length_of_top10 = []
for i in range(len(county_list_top10)):
    length_of_top10.append({i: len(county_list_top10[i])})  

# Liste mit Counties, bei denen die Länge = 10    
index = []
for i in range(len(length_of_top10)):
    if length_of_top10[i][i] == 10:
        index.append(list(length_of_top10[i].keys())[0])
    else:
        pass

# Liste mit allen Counties mit Länge = 10 zum iterieren 
counties_i2 = county_count['county'].iloc[index].tolist()

# Liste mit den Top10-Produkten pro County mit zehn Produkten
county_list_top10 = []
for county in counties_i2:
    county = df_product_count_s[(df_product_count_s['county']==county)&(df_product_count_s['ranking_in_products'].isin(np.arange(1,11)))].sort_values(by='ranking_in_products').reset_index().drop(axis=1, labels='index')
    county_list_top10.append(county)

In [ ]:
import itertools
similar_index = []
county1 = []
county2 = []
i1 = []
i2 = []
index = range(len(county_list_top10))
for a, b in itertools.combinations(index, 2):
    similar_index.append(((county_list_top10[a]['product_id'] == county_list_top10[b]['product_id']).sum()/10) ) # Top10-Produkte werden verglichen
    county1.append(str(county_list_top10[a]['county'].unique())[2:-2])
    i1.append(a)
    county2.append(str(county_list_top10[b]['county'].unique())[2:-2])
    i2.append(b)

In [ ]:
# DataFrame mit zwei Counties und deren similar_index
similar_counties = pd.DataFrame(data={'county_1': county1, 'index1': i1, 'county_2': county2, 'index2': i2, 'similar_index': similar_index})

In [ ]:
similar_counties.sort_values(by='similar_index', ascending=False).head(15)

Diese Tabelle zeigt die 15 Countypaare an, die sich am stärksten ähneln.

## Anderer Ansatz (K-Nearest-Neighbors)

In [ ]:
# DataFrame mit allen Top10-Produkten pro County
df_t = df_product_count_s[df_product_count_s['ranking_in_products'].isin(np.arange(1,11))]
df_t

In [ ]:
# Index für jedes County
county_idx = pd.DataFrame(df['county'].value_counts()).reset_index().rename(columns={'index': 'county', 'county': 'count'}).reset_index().rename(columns={'index': 'county_id'})
county_idx = county_idx.drop(axis=1, labels='count')
county_idx['county_id'] = county_idx['county_id'] + 1

In [ ]:
# neuer Index für jedes Produkt, dass in den Top10 vorkommt
product_idx = pd.DataFrame(df_t['product_id'].value_counts()).reset_index().rename(columns={'index': 'product', 'product_id': 'count'}).reset_index().rename(columns={'index': 'n_product_id'})
product_idx = product_idx.drop(axis=1, labels='count')
product_idx['n_product_id'] = product_idx['n_product_id'] + 1

In [ ]:
# neue Indexe an DF joinen
df_t = df_t.merge(county_idx, on='county')
df_t = df_t.merge(product_idx, left_on='product_id', right_on='product')

In [ ]:
df_t.head(20)

In [ ]:
from scipy.sparse import coo_matrix

# Erstellen der Matrix

n_rows = len(df_t.county_id.unique())
n_cols = len(df_t.n_product_id.unique())

row = df_t["county_id"] - 1
col = df_t["n_product_id"] - 1
data = df_t["ranking_in_products"]
county_product_matrix_s = coo_matrix((data, (row, col)), shape=(n_rows, n_cols))
county_product_matrix = county_product_matrix_s.toarray()

In [ ]:
# Für bessere verarbeitung wird Matrix als DF gespeichert
county_product_matrix = pd.DataFrame(county_product_matrix)
county_product_matrix = county_product_matrix.mask(county_product_matrix == 0)

In [ ]:
# Distanzen zwischen Counties werden berechnet
dist = 1 - county_product_matrix.T.corr()
dist = np.abs(dist.fillna(2)).values # wenn NaN-Wert, größt mögliche Distanz (also 2)

In [ ]:
# Map IDs zu Index
product_mapper = dict(zip(np.unique(df_t["n_product_id"]), list(range(n_cols))))
county_mapper = dict(zip(np.unique(df_t["county_id"]), list(range(n_rows))))
      
# Map Index zu IDs
product_inv_mapper = dict(zip(list(range(n_cols)), np.unique(df_t["n_product_id"])))
county_inv_mapper = dict(zip(list(range(n_rows)), np.unique(df_t["county_id"])))

In [ ]:
from sklearn.neighbors import NearestNeighbors
def find_similar_counties(county_id, X, k, metric='precomputed', show_distance=False):
      
    neighbour_ids = []
      
    county_ind = county_mapper[county_id]
    county_vec = X[county_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="auto", metric=metric)
    kNN.fit(X)
    county_vec = county_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(county_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(county_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:
county_names = dict(zip(county_idx['county_id'], county_idx['county']))

Hier kann man nun eingeben, zu welchem County man die fünf nächsten Nachbarn angezeigt haben will

In [ ]:
county_id = 1
similar_ids = find_similar_counties(county_id, dist, 5)
county_name = county_names[county_id]
print(f"Ähnliche Counties von: {county_name}")
for i in similar_ids:
    print(county_names[i])

# Random Forrest Modell (zu Aufgabe P.4)

Hier werden zwei neue Spalten berechnet für das Reorder Verhalten der User. Da diese zu berechnen aber sehr lange dauert haben wir uns dazu entschieden, diese als Dateien zu speichern und bei bedarf einzulesen um Zeit zu sparen

In [ ]:
# grouped_df = df.groupby(['user_id', 'order_number'])
# results = []

# for name, group in grouped_df:
#     n = group[group['reordered'] == True].count()[0]
#     N = group.count()[0]
#     percentage = n/N
#     results.append({'user_id': name[0], 'order_number': name[1], 'percentage_reordered': percentage})

# result_df = pd.DataFrame(results)

# grouped_df2 = df.groupby('user_id')
# results2 = []


# for name, group in grouped_df2:
#     n = group[group['reordered'] == True].count()[0]
#     N = group.count()[0]
#     percentage = n/N
#     results2.append({'user_id': name, 'percentage_reordered': percentage})

# result_df2 = pd.DataFrame(results2)

Ab hier wird das Random Forest Modell vorbereitet, trainiert, getestet und abschließen die Metriken zur Überprüfung des Modells (Accuracy, Precision, Recall) ausgegeben um zu schauen wie gut das Modell vorhersagen konnte, ob es bei einer Bestellung Trinkgeld geben wird. Als letztens wurde dann noch die Feature Importance ausgegeben, um nachzuvollziehen welche Attribute / Parameter, dass Modell am stärksten für diese Vorhersage gewichtet hat.

In [ ]:
# result_df = pd.read_csv("result_df.csv")
# result_df2= pd.read_csv("result2_df.csv")
# result_df2.rename(columns={'percentage_reordered':'percentage_reordered_full'}, inplace=True)
# df = df.merge(result_df)
# df = df.merge(result_df2)

# X = df.copy()
# X = X.drop(axis=1, labels=['product_name', 'department', 'aisle'], errors='ignore')
# y = X.pop('tip')

# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import LabelEncoder

# imp = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
# imp = imp.fit(X[['days_since_prior_order']])

# X['days_since_prior_order'] = imp.transform(X[['days_since_prior_order']])

# encoder = LabelEncoder()
# encoder = encoder.fit(X['county'])
# X['county'] = encoder.transform(X['county'])

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# from sklearn.ensemble import RandomForestClassifier 
# rf = RandomForestClassifier(n_estimators=500, max_depth=20, min_samples_split=10)

# rf.fit(X_train, y_train)

# y_pred_train = rf.predict(X_train)
# y_pred_test = rf.predict(X_test)

# from sklearn.metrics import accuracy_score, recall_score, precision_score

# print(f"{'Accuracy Training':17} {accuracy_score(y_train, y_pred_train):10.5f}, {'Accuracy Test':17} {accuracy_score(y_test, y_pred_test):10.5f}")
# print(f"{'Recall Training':17} {recall_score(y_train, y_pred_train):10.5f}, {'Recall Test':17} {recall_score(y_test, y_pred_test):10.5f}") 
# print(f"{'Precision Training':17} {precision_score(y_train, y_pred_train):10.5f}, {'Precision Test':17} {precision_score(y_test, y_pred_test):10.5f}") 

# ## Feature importance 
# feature_col_tree = list(X.columns)

# plt.figure(figsize=(15,10))
# importance = rf.feature_importances_
# idxs = np.argsort(importance)
# plt.title("Feature Importance")
# plt.barh(range(len(idxs)),importance[idxs],align="center")
# plt.yticks(range(len(idxs)),[feature_col_tree[i] for i in idxs])
# plt.xlabel("Random Forest Feature Importance")
# plt.tight_layout()
# plt.show()

![alt text](Bilder\ML_RF_Accuracy.png "Screenshot")

![alt text](Bilder\output2.png "Screenshot")